### CHEME 5660: Calculating the Price of Call and Put Options using a Binomial Lattice Model

### Binomial lattice model
A binomial lattice model assumes that each discrete time increment, the state of the system, e.g., the share price of equity, the spot rate, etc., can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$. Different models can be developed for the specific values of the tuple $(u,d,p)$. One particular model is the Cox, Ross, and Rubinstein (CRR) model:

* [Cox, J. C.; Ross, S. A.; Rubinstein, M. (1979). "Option pricing: A simplified approach". Journal of Financial Economics. 7 (3): 229. CiteSeerX 10.1.1.379.7582. doi:10.1016/0304-405X(79)90015-1](https://www.sciencedirect.com/science/article/pii/0304405X79900151?via%3Dihub)

#### Cox, Ross and Rubinstein (CRR) model
The [CRR binomial lattice model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) was initially developed for options pricing in 1979. However, one of the critical aspects of estimating an option’s price is calculating the underlying asset’s share price. Thus, let's use the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) to compute the share price of a stock, Advanced Micro Devices, Inc, with the ticker symbol [AMD](https://finance.yahoo.com/quote/AMD?.tsrc=applewf). In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model, the `up` and `down` moves are symmetric:

$$ud = 1$$

where the magnitude of an `up` move $u$ is given by:

$$u = \exp(\sigma\sqrt{\Delta{T}})$$

The quantity $\sigma$ denotes a _volatility parameter_, and $\Delta{T}$ represents the time step. The probability $p$ of an `up` move in a [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) is given by:

$$p = \frac{\exp(\mu\Delta{T}) - d}{u - d}$$

where $\mu$ denotes a _return parameter_. In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model paradigm, the return parameter $\mu$ and the volatility parameter $\sigma$ take on common values:
* The return parameter $\mu$ is a _risk-free_ rate of return; the _risk-free_ rate $\bar{r}$ can be approximated by the [yield on T = 10-year United States Treasury debt security](https://ycharts.com/indicators/10_year_treasury_rate). 
* The volatility parameter $\sigma$ is the [implied volatility](https://www.investopedia.com/terms/i/iv.asp); the implied volatility is the market's view of the likelihood of changes in a given security's price.

### Lab setup
The code block below installs (and loads) any [Julia](https://julialang.org) packages that we need to complete the calculations.

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-5-CRR-American-Option-Pricing`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-5-CRR-American-Option-Pricing/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-5-CRR-American-Option-Pricing/Manifest.toml`


In [2]:
# packages that we need -
using PQEcolaPoint
using DataFrames
using CSV
using Statistics
using Dates

# paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT,"data");

In [3]:
df = CSV.read(joinpath(_PATH_TO_DATA,"AMD-DTE-58-options-exp-2022-12-16-monthly-near-the-money-stacked-10-18-2022.csv"), DataFrame);

### Sample CRR model calculation

In [4]:
# Setup some constants -
B = 365.0    # Days in a year (all values are per year)
DTE = 58.0   # Days to expiration
μ = 0.04065  # risk free rate: https://www.cnbc.com/quotes/US10Y
L = 100;     # number of levels on the tree
Sₒ = 57.92;  # AMD close price on 10/18/22

In [5]:
# Set the volatility -
IV = 57.3    # AMD implied volatility for K = 62 USD/share Put
σₘ = (IV/100.0);

In [6]:
# build lattice model -
sample_lattice_model = build(CRRLatticeModel; number_of_levels=(L+1), Sₒ = Sₒ, σ = σₘ, μ = μ, T = (DTE/B));

In [7]:
# what is the contract?
sample_put_contract = PutContractModel()
sample_put_contract.ticker = "AMD"
sample_put_contract.expiration_date = Date(2022, 12, 16)
sample_put_contract.strike_price = 65.0
sample_put_contract.premium = 0.0
sample_put_contract.number_of_contracts = 1
sample_put_contract.direction = 1

# what is the premimum?
𝒫 = premium(sample_put_contract, sample_lattice_model)

# print out the premium -
println("The premium equals 𝒫 = $(round(𝒫, digits=2)) USD/share")

The premium equals 𝒫 = 9.57 USD/share


### Calculations

#### a) How does option premium vary with the strike price?
__Todo__: select the `Put` contracts from the [DataFrame](https://dataframes.juliadata.org/stable/) `df` and simulate all the strikes using a `CRR` model.

* Use the [filter](https://dataframes.juliadata.org/stable/lib/functions/#Base.filter) command to select rows from a [DataFrame](https://dataframes.juliadata.org/stable/).
* Use the [nrow](https://dataframes.juliadata.org/stable/lib/functions/#DataAPI.nrow) command get the number of rows in a [DataFrame](https://dataframes.juliadata.org/stable/).
* Use the index notation, i.e., `df[i,col::Symbol]` to select a particular value. For example, the command `df[3,:IV]` selects the `IV` value from row 3. 

#### b) How does option premium vary with the length of the contract?
__Todo__: Compare the price of `Put` contracts (with the same strikes) for T = 30 days to expiration (DTE) versus T = 58 days to expiration.

In [8]:
# load the DTE = 30 data set -
df_DTE_30 = CSV.read(joinpath(_PATH_TO_DATA,"AMD-DTE-30-options-exp-2022-11-18-monthly-near-the-money-stacked-10-18-2022.csv"), DataFrame);

#### c) How does option premium vary with the Implied Volatility (IV)?
__Todo__: Select a `Put` contract from the [DataFrame](https://dataframes.juliadata.org/stable/) `df` and simulate what happens to the price if we increase, or decrease the IV (all else held constant).

#### d) Live option price calculation
Let's select a ticker, contract type, expiration and a strike price and compare the CRR model calculation against live data.

In [9]:
# Steps -

# setup constants -
# ...

# Build a contract model -
# ...

# Build a lattice model -
# ...

# Calculate the premium -
# ...